Exercises created by Qwen2.5-Max

In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import time

### Tensor Manipulation

1.1 Modify the code to load a different dataset, such as CIFAR-10 (torchvision.datasets.CIFAR10). <br>
1.2 Change the batch size (batch_size) to 32 and observe how this affects the training time. <br>
1.3 Add a custom transformation to the data pipeline, such as normalizing pixel values to the range [-1, 1]. <br>
     

In [4]:
# ToTensor transforms from (0, 255) to (0, 1), but i want to transform it to (-1, 1),
# which is better to work with activation functions such as Tahn, while sigmoid is better
# with (0, 1).

# this refers to the value a pixel can take, where (0, 255) represents a normal representation
# most images format, where 0 is black and 255 white.

# creating a transformation function

transform = transforms.Compose([
    transforms.ToTensor(), # normalise to [0, 1]
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # normalise to [-1, 1]
])

# Download training data from open datasets.
training_data = datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=transform
)

# Download test data from open datasets.
test_data = datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=transform
)

# modifying the batch size to to 32 e compare o tempo de treino com 64

batch_size_64 = 64
batch_size_32 = 32

# train dataloaders
train_dataloader_64 = DataLoader(training_data, batch_size=batch_size_64)
train_dataloader_32 = DataLoader(training_data, batch_size=batch_size_32)

# test dataloaders
test_dataloader_64 = DataLoader(test_data, batch_size=batch_size_64)
test_dataloader_32 = DataLoader(test_data, batch_size=batch_size_32)

# find out what the shape necessary for the input
for X, y in train_dataloader_64:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 3, 32, 32])
Shape of y: torch.Size([64]) torch.int64


In [6]:
# making sure it runs on gpu
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"

# defining model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(3*32*32, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

# train loop
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # Computer prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

epochs = 5

# training loop with batch_size 64
start_test1 = time.time() # save initial time
for t in range(epochs):
    print(f"Epoch dataloader_64 {t+1}\n--------------")
    train(train_dataloader_64, model, loss_fn, optimizer)
end_test1 = time.time() # save end time

# training loop with batch_size 32
start_test2 = time.time()
for t in range(epochs):
    print(f"Epoch dataloader_32 {t+1}\n--------------")
    train(train_dataloader_32, model, loss_fn, optimizer)
end_test2 = time.time()

print(f"Time elapsed for batch_size of 64:{end_test1-start_test1:5f}")
print(f"Time elapsed for batch_size of 32:{end_test2-start_test2:5f}")

Epoch dataloader_64 1
--------------
loss: 2.293588 [   64/50000]
loss: 2.296496 [ 6464/50000]
loss: 2.285605 [12864/50000]
loss: 2.276576 [19264/50000]
loss: 2.282355 [25664/50000]
loss: 2.269580 [32064/50000]
loss: 2.270891 [38464/50000]
loss: 2.246101 [44864/50000]
Epoch dataloader_64 2
--------------
loss: 2.246453 [   64/50000]
loss: 2.232383 [ 6464/50000]
loss: 2.214886 [12864/50000]
loss: 2.229003 [19264/50000]
loss: 2.201321 [25664/50000]
loss: 2.203708 [32064/50000]
loss: 2.204005 [38464/50000]
loss: 2.163888 [44864/50000]
Epoch dataloader_64 3
--------------
loss: 2.193636 [   64/50000]
loss: 2.162617 [ 6464/50000]
loss: 2.128359 [12864/50000]
loss: 2.175662 [19264/50000]
loss: 2.113619 [25664/50000]
loss: 2.135772 [32064/50000]
loss: 2.143578 [38464/50000]
loss: 2.080114 [44864/50000]
Epoch dataloader_64 4
--------------
loss: 2.138625 [   64/50000]
loss: 2.098900 [ 6464/50000]
loss: 2.045592 [12864/50000]
loss: 2.126187 [19264/50000]
loss: 2.043897 [25664/50000]
loss: 2.082

### Model Definition

2.1 Modify the model to include an additional hidden layer with 128 neurons and a ReLU activation function. <br>
2.2 Replace the ReLU activation function with another, such as LeakyReLU or Tanh. Compare the results. <br>


In [12]:
# define modified model
class modifiedNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_stack = nn.Sequential(
            nn.Linear(3*32*32, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.LeakyReLU(),
            nn.Linear(128, 10)
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_stack(x)
        return logits
    
model_modified = modifiedNN().to(device)


# training
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")


# test
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")


In [13]:
epoch = 5

# comparing both modified and plain model

# modified
for t in range(epochs):
    print(f"Epoch {t+1}\n---------------------")
    train(train_dataloader_64, model_modified, loss_fn, optimizer)
    test(test_dataloader_64, model_modified, loss_fn)

Epoch 1
---------------------
loss: 2.320637 [   64/50000]
loss: 2.311589 [ 6464/50000]
loss: 2.297721 [12864/50000]
loss: 2.314184 [19264/50000]
loss: 2.308749 [25664/50000]
loss: 2.306344 [32064/50000]
loss: 2.304973 [38464/50000]
loss: 2.296851 [44864/50000]
Test Error: 
 Accuracy: 9.2%, Avg loss: 2.305395

Epoch 2
---------------------
loss: 2.320637 [   64/50000]
loss: 2.311589 [ 6464/50000]
loss: 2.297721 [12864/50000]
loss: 2.314184 [19264/50000]
loss: 2.308749 [25664/50000]
loss: 2.306344 [32064/50000]
loss: 2.304973 [38464/50000]
loss: 2.296851 [44864/50000]
Test Error: 
 Accuracy: 9.2%, Avg loss: 2.305395

Epoch 3
---------------------
loss: 2.320637 [   64/50000]
loss: 2.311589 [ 6464/50000]
loss: 2.297721 [12864/50000]
loss: 2.314184 [19264/50000]
loss: 2.308749 [25664/50000]
loss: 2.306344 [32064/50000]
loss: 2.304973 [38464/50000]
loss: 2.296851 [44864/50000]
Test Error: 
 Accuracy: 9.2%, Avg loss: 2.305395

Epoch 4
---------------------
loss: 2.320637 [   64/50000]
loss:

In [14]:
epoch = 5

# comparing both modified and plain model

# plain
for t in range(epochs):
    print(f"Epoch {t+1}\n---------------------")
    train(train_dataloader_64, model, loss_fn, optimizer)
    test(test_dataloader_64, model, loss_fn)

Epoch 1
---------------------
loss: 1.747504 [   64/50000]
loss: 1.718490 [ 6464/50000]
loss: 1.488520 [12864/50000]
loss: 1.800623 [19264/50000]
loss: 1.740206 [25664/50000]
loss: 1.805503 [32064/50000]
loss: 1.844375 [38464/50000]
loss: 1.671431 [44864/50000]
Test Error: 
 Accuracy: 40.1%, Avg loss: 1.713593

Epoch 2
---------------------
loss: 1.721497 [   64/50000]
loss: 1.693944 [ 6464/50000]
loss: 1.467497 [12864/50000]
loss: 1.786323 [19264/50000]
loss: 1.723432 [25664/50000]
loss: 1.790113 [32064/50000]
loss: 1.825862 [38464/50000]
loss: 1.658101 [44864/50000]
Test Error: 
 Accuracy: 40.4%, Avg loss: 1.699543

Epoch 3
---------------------
loss: 1.700364 [   64/50000]
loss: 1.671583 [ 6464/50000]
loss: 1.447907 [12864/50000]
loss: 1.773197 [19264/50000]
loss: 1.707784 [25664/50000]
loss: 1.775854 [32064/50000]
loss: 1.807949 [38464/50000]
loss: 1.645206 [44864/50000]
Test Error: 
 Accuracy: 40.8%, Avg loss: 1.686335

Epoch 4
---------------------
loss: 1.681139 [   64/50000]
lo

### Treinamento do Modelo  

3.1 Altere o número de épocas para 10 e observe como isso afeta a precisão do modelo.<br>

In [15]:
epoch = 10
# comparing how different epoch sizes affect the model
# plain
for t in range(epochs):
    print(f"Epoch {t+1}\n---------------------")
    train(train_dataloader_64, model, loss_fn, optimizer)
    test(test_dataloader_64, model, loss_fn)

Epoch 1
---------------------
loss: 1.647398 [   64/50000]
loss: 1.615947 [ 6464/50000]
loss: 1.396326 [12864/50000]
loss: 1.739592 [19264/50000]
loss: 1.665690 [25664/50000]
loss: 1.739096 [32064/50000]
loss: 1.758780 [38464/50000]
loss: 1.610207 [44864/50000]
Test Error: 
 Accuracy: 41.9%, Avg loss: 1.651479

Epoch 2
---------------------
loss: 1.632709 [   64/50000]
loss: 1.600811 [ 6464/50000]
loss: 1.381337 [12864/50000]
loss: 1.730325 [19264/50000]
loss: 1.653200 [25664/50000]
loss: 1.728236 [32064/50000]
loss: 1.743729 [38464/50000]
loss: 1.599721 [44864/50000]
Test Error: 
 Accuracy: 42.4%, Avg loss: 1.641216

Epoch 3
---------------------
loss: 1.619340 [   64/50000]
loss: 1.587145 [ 6464/50000]
loss: 1.367351 [12864/50000]
loss: 1.721909 [19264/50000]
loss: 1.641701 [25664/50000]
loss: 1.718011 [32064/50000]
loss: 1.729756 [38464/50000]
loss: 1.589820 [44864/50000]
Test Error: 
 Accuracy: 42.8%, Avg loss: 1.631541

Epoch 4
---------------------
loss: 1.607414 [   64/50000]
lo

In [16]:
epoch = 20
# comparing how different epoch sizes affect the model
# plain
for t in range(epochs):
    print(f"Epoch {t+1}\n---------------------")
    train(train_dataloader_64, model, loss_fn, optimizer)
    test(test_dataloader_64, model, loss_fn)

Epoch 1
---------------------
loss: 1.586795 [   64/50000]
loss: 1.552444 [ 6464/50000]
loss: 1.330564 [12864/50000]
loss: 1.703293 [19264/50000]
loss: 1.612178 [25664/50000]
loss: 1.690472 [32064/50000]
loss: 1.691673 [38464/50000]
loss: 1.563648 [44864/50000]
Test Error: 
 Accuracy: 43.8%, Avg loss: 1.605417

Epoch 2
---------------------
loss: 1.577868 [   64/50000]
loss: 1.542256 [ 6464/50000]
loss: 1.319618 [12864/50000]
loss: 1.698557 [19264/50000]
loss: 1.603293 [25664/50000]
loss: 1.681903 [32064/50000]
loss: 1.679796 [38464/50000]
loss: 1.555648 [44864/50000]
Test Error: 
 Accuracy: 44.2%, Avg loss: 1.597474

Epoch 3
---------------------
loss: 1.569596 [   64/50000]
loss: 1.532812 [ 6464/50000]
loss: 1.309147 [12864/50000]
loss: 1.694107 [19264/50000]
loss: 1.594513 [25664/50000]
loss: 1.673611 [32064/50000]
loss: 1.668659 [38464/50000]
loss: 1.548197 [44864/50000]
Test Error: 
 Accuracy: 44.5%, Avg loss: 1.589854

Epoch 4
---------------------
loss: 1.561926 [   64/50000]
lo

In [17]:
epoch = 50
# comparing how different epoch sizes affect the model
# plain
for t in range(epochs):
    print(f"Epoch {t+1}\n---------------------")
    train(train_dataloader_64, model, loss_fn, optimizer)
    test(test_dataloader_64, model, loss_fn)

Epoch 1
---------------------
loss: 1.547964 [   64/50000]
loss: 1.507553 [ 6464/50000]
loss: 1.280073 [12864/50000]
loss: 1.681130 [19264/50000]
loss: 1.569011 [25664/50000]
loss: 1.650088 [32064/50000]
loss: 1.638507 [38464/50000]
loss: 1.527525 [44864/50000]
Test Error: 
 Accuracy: 45.2%, Avg loss: 1.568628

Epoch 2
---------------------
loss: 1.541721 [   64/50000]
loss: 1.499857 [ 6464/50000]
loss: 1.271465 [12864/50000]
loss: 1.676697 [19264/50000]
loss: 1.560723 [25664/50000]
loss: 1.642680 [32064/50000]
loss: 1.629897 [38464/50000]
loss: 1.520763 [44864/50000]
Test Error: 
 Accuracy: 45.4%, Avg loss: 1.562034

Epoch 3
---------------------
loss: 1.535698 [   64/50000]
loss: 1.492443 [ 6464/50000]
loss: 1.263055 [12864/50000]
loss: 1.672385 [19264/50000]
loss: 1.552816 [25664/50000]
loss: 1.635186 [32064/50000]
loss: 1.621818 [38464/50000]
loss: 1.514438 [44864/50000]
Test Error: 
 Accuracy: 45.5%, Avg loss: 1.555633

Epoch 4
---------------------
loss: 1.530040 [   64/50000]
lo